### Global Yearly Averages
#### This notebook grabs the monthly salinity rasters that I downloaded and generates yearly averages.

#### Group monthly rasters by year and calculate annual means
##### Input: Monthly rasters named SSS_YYYYMM (2011-2024)
##### Output: 14 yearly average rasters (2011-2024)

In [36]:
import arcpy
import os
from arcpy.sa import *
import re

# Setup
arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput = True
arcpy.env.workspace = r"C:\Users\Liliana\Documents\ArcGIS\Projects\SalinityDrafting\Salinity_Analysis.gdb"

In [39]:
# List all monthly rasters
all_rasters = arcpy.ListRasters("SSS_*")
print(f"Found {len(all_rasters)} monthly rasters")

# Extract year-month from raster names and organize by year
monthly_by_year = {}

for raster in all_rasters:
    # Extract YYYYMM from raster name (assuming format: SSS_YYYYMM)
    match = re.search(r'SSS_(\d{6})', raster)
    if match:
        yyyymm = match.group(1)
        year = yyyymm[:4]  # First 4 digits = year
        month = yyyymm[4:]  # Last 2 digits = month
        
        # Add to dictionary
        if year not in monthly_by_year:
            monthly_by_year[year] = []
        monthly_by_year[year].append(raster)

Found 161 monthly rasters


In [40]:
# Show found data
print(f"\nFound data for {len(monthly_by_year)} years:")
for year in sorted(monthly_by_year.keys()):
    month_count = len(monthly_by_year[year])
    print(f"  {year}: {month_count} months")


Found data for 14 years:
  2011: 3 months
  2012: 12 months
  2013: 12 months
  2014: 12 months
  2015: 12 months
  2016: 12 months
  2017: 12 months
  2018: 12 months
  2019: 12 months
  2020: 12 months
  2021: 12 months
  2022: 12 months
  2023: 12 months
  2024: 12 months


In [41]:
# Calculate yearly averages
yearly_rasters = []

for year in sorted(monthly_by_year.keys()):
    monthly_list = monthly_by_year[year]
    
    # Check if complete or partial year
    if len(monthly_list) >= 1:  # Process partial years
        print(f"\nProcessing {year} ({len(monthly_list)} months)...")
        
        # Create yearly average
        yearly_raster_name = f"SSS_Yearly_{year}"
        print(f"  Calculating {yearly_raster_name}...")
        
        # Use CellStatistics to calculate mean across months
        yearly_mean = CellStatistics(monthly_list, "MEAN", "DATA")
        
        # Save the yearly average
        yearly_mean.save(yearly_raster_name)
        yearly_rasters.append(yearly_raster_name)
        
        print(f"  ✓ Saved: {yearly_raster_name}")
        
        # Get basic statistics
        try:
            min_val = arcpy.management.GetRasterProperties(yearly_raster_name, "MINIMUM")
            max_val = arcpy.management.GetRasterProperties(yearly_raster_name, "MAXIMUM")
            mean_val = arcpy.management.GetRasterProperties(yearly_raster_name, "MEAN")
            
            print(f"    Stats: {float(min_val.getOutput(0)):.1f}-{float(max_val.getOutput(0)):.1f} PSU")
            print(f"    Mean: {float(mean_val.getOutput(0)):.2f} PSU")
        except:
            print(f"    Stats: (unavailable)")

print(f"\n✓ Created {len(yearly_rasters)} yearly average rasters")


Processing 2011 (3 months)...
  Calculating SSS_Yearly_2011...
  ✓ Saved: SSS_Yearly_2011
    Stats: 5.4-40.4 PSU
    Mean: 34.40 PSU

Processing 2012 (12 months)...
  Calculating SSS_Yearly_2012...
  ✓ Saved: SSS_Yearly_2012
    Stats: 5.1-40.2 PSU
    Mean: 34.36 PSU

Processing 2013 (12 months)...
  Calculating SSS_Yearly_2013...
  ✓ Saved: SSS_Yearly_2013
    Stats: 5.1-40.3 PSU
    Mean: 34.43 PSU

Processing 2014 (12 months)...
  Calculating SSS_Yearly_2014...
  ✓ Saved: SSS_Yearly_2014
    Stats: 5.3-39.9 PSU
    Mean: 34.42 PSU

Processing 2015 (12 months)...
  Calculating SSS_Yearly_2015...
  ✓ Saved: SSS_Yearly_2015
    Stats: 5.4-39.9 PSU
    Mean: 34.38 PSU

Processing 2016 (12 months)...
  Calculating SSS_Yearly_2016...
  ✓ Saved: SSS_Yearly_2016
    Stats: 5.5-39.8 PSU
    Mean: 34.41 PSU

Processing 2017 (12 months)...
  Calculating SSS_Yearly_2017...
  ✓ Saved: SSS_Yearly_2017
    Stats: 5.9-39.9 PSU
    Mean: 34.38 PSU

Processing 2018 (12 months)...
  Calculating SSS